In [ ]:
import dill

In [ ]:
dill.load_session('user_and_course_dfs.db')

In [ ]:
import itertools
from itertools import combinations 
from utilities import *

In [ ]:
%run heatmap_utilities.ipynb
%run other_graphing_utilities.ipynb
c.to_file = False

In [ ]:
def plot_transition_heatmap(data, *, figsize, font_scale=1, 
                            title_name="", title_addition="", 
                            transpose=False, tick_label_colors=None,
                            width_ratios=None, quantile=True, vmax=None,
                            all_resources=False, int_tick_labels=False,
                            no_title=False):
    
    func = get_counts_all_resources if all_resources else get_counts
    
    plot_2d_values_heatmap(data, func=func, figsize=figsize, font_scale=font_scale, 
                           title_name=title_name, title_addition=title_addition,
                           xlabel='Origin Resources - "From"',
                           ylabel='Destination Resources - "To"', 
                           transpose=transpose, tick_label_colors=tick_label_colors,
                           width_ratios=width_ratios, quantile=quantile, vmax=vmax,
                           int_tick_labels=int_tick_labels, no_title=no_title)
    
def broken_y_bar_histogram_df(flattened, bins):
    # TODO: make this simpler wiithout two calls to np.histogram
    vals, divisions = map(list, np.histogram(flattened, bins=bins))
    divisions = [int(d) for d in divisions]
    vals, divisions = map(list, np.histogram(flattened, bins=divisions))
    
    # for some reason, numpy returns zero in its values for the histogram function...
    i = 0

    while i < len(vals):
        if not vals[i]:
            del vals[i]
            del divisions[i]
            i -= 1

        i += 1
        
    vals.append(0)

    return pd.DataFrame(zip(divisions, vals)).set_index(0)


def get_transversal_frequency_hist_df(data, bins):
    _, freqs = get_counts(data)
    flattened = list(itertools.chain.from_iterable(freqs))

    return broken_y_bar_histogram_df(flattened, bins=bins)


In [ ]:
c.to_file = False
plot_transition_heatmap(user_urls_dfs, figsize=(40,40),
                       font_scale=4.5, no_title=True,
                       int_tick_labels=True, vmax=50)

In [ ]:
mod_figs = {
    '1fa8ba0e70222969eae8a0b482b42628': (8, 8),
    '2cb63d38c0d7afccf3853ee3e8b9653e': (10, 10),
    '34710aa14b316648e90d3e97b8ca952b': (8, 8),
    '533bd6d1652f38b4426120b2e6bc4977': (10, 10),
    '5349577d9e4c7b0d0bfe3403e9be3d47': (20, 20),
    '798ae73ecfb86a78ab89be6e3ee33595': (8, 8),
    '7e9e8df5eb95867d189a107ca4706ea8': (20, 20),
    'c4b6c2c3a3f08957abc17ee5c4752e84': (20, 20),
    'fe540222fd0dfe96bcbc20d5a250e146': (20, 20)
}

plot_heatmaps(user_urls_dfs, mod_figs, plot_transition_heatmap)

In [ ]:
plot_transition_heatmap(user_urls_dfs, figsize=(40, 40), 
                        font_scale=2.5, title_name="Combined (No SCM BKY)",
                        tick_label_colors=resource_categories,
                        width_ratios=[1, 0.05])

In [ ]:
mod_figs = {
    '2cb63d38c0d7afccf3853ee3e8b9653e': (10, 10),
    '34710aa14b316648e90d3e97b8ca952b': (10, 10),
    '5349577d9e4c7b0d0bfe3403e9be3d47': (20, 20),
    '96749bbb0b4e0976c985a92bd3f0ddb0': (10, 10)
}

plot_heatmaps(ai_edx_dfs, mod_figs, plot_transition_heatmap, title_addition="with SCM and BKY")

In [ ]:
plot_transition_heatmap(ai_edx_dfs, figsize=(40, 40), font_scale=2.5, 
                        title_name="Transposed Combined", 
                        title_addition="with SCM and BKY", 
                        transpose=True, tick_label_colors=resource_categories,
                        width_ratios=[1, 0.05])

In [ ]:
plot_transition_heatmap(all_resource_dfs, figsize=(80, 70), font_scale=2.5, 
                        title_name="Transposed Combined", 
                        title_addition="with everything vmax=50", 
                        transpose=True, tick_label_colors=resource_categories,
                        width_ratios=[1, 0.05], vmax=50)

In [ ]:
plot_transition_heatmap(all_resource_dfs, figsize=(80, 60), font_scale=2.5, 
                        title_name="Transposed Combined", 
                        title_addition="with everything and all resources", 
                        transpose=True,
                        width_ratios=[1, 0.05], vmax=160, all_resources=True)

In [ ]:
print(all_resource_dfs[2])

In [ ]:
print(section_of_resources['Component property'])

In [ ]:
filtered_units = []
for u in all_resource_dfs:
    filtered = []
    for _, row in u.iterrows():
        try:
            if '6' in section_of_resources[row['display_name']] or '7' in section_of_resources[row['display_name']] \
            or '8' in section_of_resources[row['display_name']] or '9' in section_of_resources[row['display_name']]:
                filtered.append(row)
        except KeyError:
            pass

    if filtered:
        filtered_units.append(pd.DataFrame(filtered))

plot_transition_heatmap(filtered_units, figsize=(80, 60), font_scale=2.5, 
                        title_name="Units filtered", 
                        title_addition="with everything and all resources", 
                        transpose=True, tick_label_colors=resource_categories,
                        width_ratios=[1, 0.05], all_resources=True)

In [ ]:
plot_transition_heatmap(filtered_units, figsize=(40, 25), font_scale=2.5, 
                        title_name="Units filtered",
                        transpose=True, tick_label_colors=resource_categories,
                        width_ratios=[1, 0.05], quantile=False)

In [ ]:
df = get_transversal_frequency_hist_df(user_urls_dfs, 5)

plot_broken_y_bar(
    df, lims=[(0, 12), (4000, 4150)], 
    xlabel="Traversal (Without SCM and BKY) Frequency", ylabel="Occurences", 
    ylabel_loc=(-2.5, 15), figsize=(3, 4), breakline_len=.024,
    width=1, align='edge'
)

In [ ]:
df = get_transversal_frequency_hist_df(ai_edx_dfs, 60)

plot_broken_y_bar(
    df, lims=[(0, 24), (4302, 4360)], 
    xlabel="Traversal (With SCM and BKY) Frequency", ylabel="Occurences", 
    ylabel_loc=(-3, 30), figsize=(3, 4), breakline_len=.024,
    width=1, align='edge'
)